# Description of the note should come here.

In [3]:
#Pkg.add("Pkg"); import Pkg; 
using Pkg
using Distributed
using DelimitedFiles
using Profile    
using StatsBase
using Random
using Statistics
using LinearAlgebra
using Plots
using Printf
using Measures
using CSV
using LaTeXStrings;
using DataFrames
rng = Random.MersenneTwister(1234);

In [4]:
include("../src/basic_MPL.jl");
include("../src/basic.jl")
include("../src/my_HIV_data_cleanign.jl")
include("../src/basic_analysis.jl")
include("../src/analysis.jl")
include("../src/non_synonymous_N-glycan.jl") 
include("../src/process_HIV_mutation_for_CSV.jl") 
NUC = ["-", "A", "C", "G", "T"];

# Processes Start

## Retrieve variable and antibody binding sites 

In [106]:
# Keep trucking only the following sites and nucleotides;
# Should include also HXB2 inde for protein and genes. 
hxb2csv = DataFrame(CSV.File("../data/csv/hxb2.csv"));
csv_raw_DH270_binding = DataFrame(CSV.File("../data/csv/DH270_binding.csv"))
csv_raw_DH272_binding = DataFrame(CSV.File("../data/csv/DH272_binding.csv"))
csv_raw_DH475_binding = DataFrame(CSV.File("../data/csv/DH475_binding.csv"))
csv_raw_CD4contact = DataFrame(CSV.File("../data/csv/CD4_contct_residue.csv"))
csv_raw_V1_HV = DataFrame(CSV.File("../data/csv/V1_Hypervariable_residue.csv"))
csv_raw_V2_HV = DataFrame(CSV.File("../data/csv/V2_Hypervariable_residue.csv"))
csv_raw_V3_HV = DataFrame(CSV.File("../data/csv/V3_Hypervariable_residue.csv"))
csv_raw_V4_HV = DataFrame(CSV.File("../data/csv/V4_Hypervariable_residue.csv"))
csv_raw_V5_HV = DataFrame(CSV.File("../data/csv/V5_Hypervariable_residue.csv"))
csv_raw_LoopD = DataFrame(CSV.File("../data/csv/Loop-D_residue.csv"));

csv_raw_resist_mut_DH270 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH270.csv")) 
csv_raw_resist_mut_DH272 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH272.csv")) 
csv_raw_resist_mut_DH475 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH475.csv")) 
csv_raw_resist_strain_specific_Abs_CH848 = DataFrame(CSV.File("../data/csv/resistant_mut_to_strain_specific-bnAbs_CH848.csv")) 
csv_raw_common_mut_SHIV_CH848 = DataFrame(CSV.File("../data/csv/common_mutation_rhesus_SHIV848.csv")) ;

## Formatting the files

In [107]:
#csv_selection = DataFrame(CSV.File("/net/dali/home/barton/kais/paper-MPL-inference-master/data/HIV/analysis/703010848-3-poly.csv"));
csv_selection = DataFrame(CSV.File("../data/mpl/703010848-3-poly.csv"));
gene_set = [[] for _ in 1:3]; vec_AA_idx = [[] for _ in 1:3]; HXB2_set = [[] for _ in 1:3]; 
vec_nuc_idx = []; frame_set = []; 
for x in csv_selection.HXB2_index
    num_nuc = get_num_nuc(x)
    frame, gene = index2frame(num_nuc)
    num_nuc, num_AA, gene = map_numNUC_to_numAA(x, frame)    
    push!(frame_set, frame)
    push!(vec_nuc_idx, num_nuc)
    for fr in 1:3 
        push!(gene_set[fr], gene[fr])
        push!(vec_AA_idx[fr], num_AA[fr])
        push!(HXB2_set[fr], HXB2(num_nuc, num_AA[fr], gene[fr]))
    end
end;

@printf("Fr.1: gene: %s\n", join(unique(gene_set[1]), ", "));
@printf("Fr.2: gene: %s\n", join(unique(gene_set[2]), ", "));
@printf("Fr.3: gene: %s\n", join(unique(gene_set[3]), ", "));
@printf("max: %d, %d\n", maximum(skipmissing(vec_nuc_idx)), maximum(skipmissing(vec_nuc_idx))) 
@printf("min: %d, %d\n", minimum(skipmissing(vec_nuc_idx)), minimum(skipmissing(vec_nuc_idx)));

# Use frame 3 where Env gene belongs. 
vec_DH270_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH270_binding)
vec_DH272_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH272_binding)
vec_DH475_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH475_binding)
vec_CD4contact_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_CD4contact)
vec_V1 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V1_HV)
vec_V2 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V2_HV)
vec_V3 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V3_HV)
vec_V4 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V4_HV)
vec_V5 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V5_HV)
vec_LoopD = get_true_false_variable_region(vec_AA_idx[3], csv_raw_LoopD);

csv_raw_resist_mut_DH270 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH270.csv")) 
csv_raw_resist_mut_DH272 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH272.csv")) 
csv_raw_resist_mut_DH475 = DataFrame(CSV.File("../data/csv/resistant_mut_to_DH475.csv")) 
csv_raw_resist_strain_specific_Abs_CH848 = DataFrame(CSV.File("../data/csv/resistant_mut_to_strain_specific-bnAbs_CH848.csv")) 
csv_raw_common_mut_SHIV_CH848 = DataFrame(CSV.File("../data/csv/common_mutation_rhesus_SHIV848.csv")) ;

# These are all waat I need. 
csv_index_and_TF= DataFrame(CSV.File("../data/mpl/703010848-3-index.csv"));
seq_num_raw = readdlm("../data/mpl/703010848-3-poly-seq2state.dat", Int);

Fr.1: gene: , vif, tat, nef
Fr.2: gene: , tat, vpu, rev
Fr.3: gene: pol, , vpr, rev, env
max: 9603, 9603
min: 4924, 4924


## Processes to get mutations 

In [108]:
(mutant_hxb2, mutant_nuc, mutant_date_found, mutant_gene, mutant_types_set_AA, 
mutant_types_set_nuc, mutant_types_set_nuc_simple, plus_glycan_set, minus_glycan_set, shifted_glycan_set
) = Make_combination_of_mutations_with_genetic_background_w_glycan(csv_index_and_TF, seq_num_raw, hxb2csv); 
mutant_types_set_AA_filtered = []
push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[1]));
push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[2]));
push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[3]));


mutant_types_set_TF_AA = get_TF_AA(csv_index_and_TF, mutant_hxb2) ;
mutant_types_set_AA_filtered_v2 = []
push!(mutant_types_set_AA_filtered_v2, replacing_redundant_AA_by_TF(mutant_types_set_AA_filtered[1], mutant_types_set_TF_AA[1]))
push!(mutant_types_set_AA_filtered_v2, replacing_redundant_AA_by_TF(mutant_types_set_AA_filtered[2], mutant_types_set_TF_AA[2]))
push!(mutant_types_set_AA_filtered_v2, replacing_redundant_AA_by_TF(mutant_types_set_AA_filtered[3], mutant_types_set_TF_AA[3]));
mutant_nuc_simple_filtered = filter_nuc_mut(mutant_types_set_nuc_simple);

## Check the entories of resistant mutations

In [109]:
# Check the ordering is the same as the *-3-poly and *-3-index. 
for i in 1:length(csv_selection.HXB2_index)
    i_HXB2 = csv_selection.HXB2_index[i]
    i_HXB2_v2 = mutant_hxb2[i]
    nuc = csv_selection.nucleotide[i]
    nuc_v2 = mutant_nuc[i]
    @assert (i_HXB2 == i_HXB2_v2 && nuc == nuc_v2)
end

# --- Check whether mutations have been reported or not. ---# 
reported_mutant_data = copy(String.(csv_raw_resist_mut_DH270[:, 1]))
bool_resist_mut_DH270 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_mut_DH270)

reported_mutant_data = copy(String.(csv_raw_resist_mut_DH272[:, 1]))
bool_resist_mut_DH272 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_mut_DH272)

reported_mutant_data = copy(String.(csv_raw_resist_mut_DH475[:, 1]))
bool_resist_mut_DH475 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_mut_DH475)

reported_mutant_data = copy(String.(csv_raw_resist_strain_specific_Abs_CH848[:, 1]))
bool_resist_strain_specific_Abs_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_resist_strain_specific_Abs_CH848)

reported_mutant_data = copy(String.(csv_raw_common_mut_SHIV_CH848[:, 1]))
bool_common_mut_SHIV_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
@show sum(bool_common_mut_SHIV_CH848);
reversion_true_false = csv_selection.consensus .== csv_selection.nucleotide .!= csv_selection.TF;

sum(bool_resist_mut_DH270) = 33
sum(bool_resist_mut_DH272) = 10
sum(bool_resist_mut_DH475) = 7
sum(bool_resist_strain_specific_Abs_CH848) = 14
sum(bool_common_mut_SHIV_CH848) = 2


In [110]:

# --- Just filling basic information ----#
idx_sort_MPL = sortperm(csv_selection.s_MPL, rev=true);
idx = (mutant_types_set_nuc[3][idx_sort_MPL] .!= "") .* (mutant_date_found[idx_sort_MPL] .> 0)
df = DataFrame(
    HXB2=vec_nuc_idx[idx_sort_MPL][idx], 
)

df[!, "i_AA_fr1"]=replace_nothing_with_missing(vec_AA_idx[1])[idx_sort_MPL][idx]
df[!, "i_AA_fr2"]=replace_nothing_with_missing(vec_AA_idx[2])[idx_sort_MPL][idx]
df[!, "i_AA_fr3"]=replace_nothing_with_missing(vec_AA_idx[3])[idx_sort_MPL][idx]
df[!, "gene_fr1"]=replace_nothing_with_missing(gene_set[1])[idx_sort_MPL][idx]
df[!, "gene_fr2"]=replace_nothing_with_missing(gene_set[2])[idx_sort_MPL][idx]
df[!, "gene_fr3"]=replace_nothing_with_missing(gene_set[3])[idx_sort_MPL][idx]

# -- Just copying fields already in the *-3-poly.csv
for header_temp in names(csv_selection)
    if(!occursin(r"f_at", header_temp) && !occursin(r"epitope", header_temp))
        df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
    end
end
# --- Writing computed variables including types of mutaions ----#
df[!, "detected_date"] = mutant_date_found[idx_sort_MPL][idx]
#df[!, "mutants_nuc_filtered"] = mutant_types_set_nuc_simple[3][idx_sort_MPL][idx]

df[!, "mutants_nuc"] = mutant_nuc_simple_filtered[idx_sort_MPL][idx]
df[!, "mutants_AA_fr1"] = mutant_types_set_AA_filtered_v2[1][idx_sort_MPL][idx]
df[!, "mutants_AA_fr2"] = mutant_types_set_AA_filtered_v2[2][idx_sort_MPL][idx]
df[!, "mutants_AA_fr3"] = mutant_types_set_AA_filtered_v2[3][idx_sort_MPL][idx]
df[!, "LoopD"] = vec_LoopD[idx_sort_MPL][idx]
df[!, "CD4BS"] = vec_CD4contact_binding[idx_sort_MPL][idx]
df[!, "V1"] = vec_V1[idx_sort_MPL][idx]
df[!, "V2"] = vec_V2[idx_sort_MPL][idx]
df[!, "V3"] = vec_V3[idx_sort_MPL][idx]
df[!, "V4"] = vec_V4[idx_sort_MPL][idx]
df[!, "V5"] = vec_V5[idx_sort_MPL][idx]
df[!, "DH270BS"] = vec_DH270_binding[idx_sort_MPL][idx]
df[!, "DH272BS"] = vec_DH272_binding[idx_sort_MPL][idx]
df[!, "DH475BS"] = vec_DH475_binding[idx_sort_MPL][idx]
df[!, "resist_mut_DH270"] = bool_resist_mut_DH270[idx_sort_MPL][idx]
df[!, "resist_mut_DH272"] = bool_resist_mut_DH272[idx_sort_MPL][idx]
df[!, "resist_mut_DH475"] = bool_resist_mut_DH475[idx_sort_MPL][idx]
df[!, "resist_strain_specific_Abs_CH848"] = bool_resist_strain_specific_Abs_CH848[idx_sort_MPL][idx]
df[!, "common_mut_SHIV_CH848"] = bool_common_mut_SHIV_CH848[idx_sort_MPL][idx]
df[!, "reversion"] = reversion_true_false[idx_sort_MPL][idx]
df[!, "N_linked_glycan_plus_fr1"] = plus_glycan_set[1][idx_sort_MPL][idx]
df[!, "N_linked_glycan_minus_fr1"] = minus_glycan_set[1][idx_sort_MPL][idx]
df[!, "N_linked_glycan_shift_fr1"] = shifted_glycan_set[1][idx_sort_MPL][idx]
df[!, "N_linked_glycan_plus_fr2"] = plus_glycan_set[2][idx_sort_MPL][idx]
df[!, "N_linked_glycan_minus_fr2"] = minus_glycan_set[2][idx_sort_MPL][idx]
df[!, "N_linked_glycan_shift_fr2"] = shifted_glycan_set[2][idx_sort_MPL][idx]
df[!, "N_linked_glycan_plus_fr3"] = plus_glycan_set[3][idx_sort_MPL][idx]
df[!, "N_linked_glycan_minus_fr3"] = minus_glycan_set[3][idx_sort_MPL][idx]
df[!, "N_linked_glycan_shift_fr3"] = shifted_glycan_set[3][idx_sort_MPL][idx]

# -- Just copying single site trajectories *-3-poly.csv
for header_temp in names(csv_selection)
    if(occursin(r"f_at", header_temp) )
        df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
    end
end
CSV.write("../out/703010848-3-poly_sorted_with_mutant.csv", df);

# CSV files for Rhesus Macaques
- Note, I need to use results of coaligned sequences to compare with CH505 and RMs. 

In [111]:
fname_key_RMs = ["RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720"];

In [112]:
for key_RM in fname_key_RMs
    @sprintf("%s\n", key_RM)
    csv_selection = DataFrame(CSV.File("../data/mpl/SHIV.CH848/" * key_RM * "-3-poly.csv"));

    gene_set = [[] for _ in 1:3]; vec_AA_idx = [[] for _ in 1:3]; HXB2_set = [[] for _ in 1:3]; 
    vec_nuc_idx = []; frame_set = []; 
    for x in csv_selection.HXB2_index
        num_nuc = get_num_nuc(x)
        frame, gene = index2frame(num_nuc)
        num_nuc, num_AA, gene = map_numNUC_to_numAA(x, frame)
        #@assert(gene==gene_check)

        push!(frame_set, frame)
        push!(vec_nuc_idx, num_nuc)
        for fr in 1:3 
            push!(gene_set[fr], gene[fr])
            push!(vec_AA_idx[fr], num_AA[fr])
            push!(HXB2_set[fr], HXB2(num_nuc, num_AA[fr], gene[fr]))
        end
    end;

    @printf("Fr.1: gene: %s\n", join(unique(gene_set[1]), ", "));
    @printf("Fr.2: gene: %s\n", join(unique(gene_set[2]), ", "));
    @printf("Fr.3: gene: %s\n", join(unique(gene_set[3]), ", "));
    @printf("max: %d, %d\n", maximum(skipmissing(vec_nuc_idx)), maximum(skipmissing(vec_nuc_idx))) 
    @printf("min: %d, %d\n", minimum(skipmissing(vec_nuc_idx)), minimum(skipmissing(vec_nuc_idx)));

    # Use frame 3 where Env gene belongs. 
    vec_DH270_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH270_binding)
    vec_DH272_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH272_binding)
    vec_DH475_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_DH475_binding)

    vec_CD4contact_binding = get_true_false_variable_region(vec_AA_idx[3], csv_raw_CD4contact)
    vec_V1 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V1_HV)
    vec_V2 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V2_HV)
    vec_V3 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V3_HV)
    vec_V4 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V4_HV)
    vec_V5 = get_true_false_variable_region(vec_AA_idx[3], csv_raw_V5_HV)
    vec_LoopD = get_true_false_variable_region(vec_AA_idx[3], csv_raw_LoopD);
    # ----------------------------------------------------------------#

    # These are all waat I need. 
    csv_index_and_TF= DataFrame(CSV.File("../data/mpl/SHIV.CH848/" *key_RM* "-3-index.csv"));
    seq_num_raw = readdlm("../data/mpl/SHIV.CH848/" *key_RM* "-3-poly-seq2state.dat", Int);

    # ---- Get mutations ------#
    (mutant_hxb2, mutant_nuc, mutant_date_found, mutant_gene, mutant_types_set_AA, 
    mutant_types_set_nuc, mutant_types_set_nuc_simple, plus_glycan_set, minus_glycan_set, shifted_glycan_set
    ) = Make_combination_of_mutations_with_genetic_background_w_glycan(csv_index_and_TF, seq_num_raw, hxb2csv); 
    mutant_types_set_AA_filtered = []
    push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[1]));
    push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[2]));
    push!(mutant_types_set_AA_filtered, filtering_mutations_AA(mutant_types_set_AA[3]));
    mutant_nuc_simple_filtered = filter_nuc_mut(mutant_types_set_nuc_simple);
    # ----------------------------------------------------------------#
    # Check the ordering is the same as the *-3-poly and *-3-index. 
    
    for i in 1:length(csv_selection.HXB2_index)
        i_HXB2 = csv_selection.HXB2_index[i]
        i_HXB2_v2 = mutant_hxb2[i]
        nuc = csv_selection.nucleotide[i]
        nuc_v2 = mutant_nuc[i]
    end

    # --- Check whether mutations have been reported or not. ---# 
    reported_mutant_data = copy(String.(csv_raw_resist_mut_DH270[:, 1]))
    bool_resist_mut_DH270 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_mut_DH270)

    reported_mutant_data = copy(String.(csv_raw_resist_mut_DH272[:, 1]))
    bool_resist_mut_DH272 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_mut_DH272)

    reported_mutant_data = copy(String.(csv_raw_resist_mut_DH475[:, 1]))
    bool_resist_mut_DH475 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_mut_DH475)

    reported_mutant_data = copy(String.(csv_raw_resist_strain_specific_Abs_CH848[:, 1]))
    bool_resist_strain_specific_Abs_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_resist_strain_specific_Abs_CH848)

    reported_mutant_data = copy(String.(csv_raw_common_mut_SHIV_CH848[:, 1]))
    bool_common_mut_SHIV_CH848 = [check_mutant_is_in_reported(my_replace_nothing(string(x)), reported_mutant_data) for x in mutant_types_set_AA[3]]
    @show sum(bool_common_mut_SHIV_CH848);
    reversion_true_false = csv_selection.consensus .== csv_selection.nucleotide .!= csv_selection.TF;    
    # --- Just filling basic information ----#
    idx_sort_MPL = sortperm(csv_selection.s_MPL, rev=true);
    idx = (mutant_types_set_nuc[3][idx_sort_MPL] .!= "") .* (mutant_date_found[idx_sort_MPL] .> 0)
    df = DataFrame(HXB2=vec_nuc_idx[idx_sort_MPL][idx], )
    
    df[!, "i_AA_fr1"]=replace_nothing_with_missing(vec_AA_idx[1])[idx_sort_MPL][idx]
    df[!, "i_AA_fr2"]=replace_nothing_with_missing(vec_AA_idx[2])[idx_sort_MPL][idx]
    df[!, "i_AA_fr3"]=replace_nothing_with_missing(vec_AA_idx[3])[idx_sort_MPL][idx]
    df[!, "gene_fr1"]=replace_nothing_with_missing(gene_set[1])[idx_sort_MPL][idx]
    df[!, "gene_fr2"]=replace_nothing_with_missing(gene_set[2])[idx_sort_MPL][idx]
    df[!, "gene_fr3"]=replace_nothing_with_missing(gene_set[3])[idx_sort_MPL][idx]

    # -- Just copying fields already in the *-3-poly.csv
    for header_temp in names(csv_selection)
        if(!occursin(r"f_at", header_temp) && !occursin(r"epitope", header_temp))
            df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
        end
    end

    # --- Writing computed variables including types of mutaions ----#
    df[!, "detected_date"] = mutant_date_found[idx_sort_MPL][idx]
#    df[!, "mutants_nuc_filtered"] = mutant_types_set_nuc_simple[3][idx_sort_MPL][idx]
    df[!, "mutants_nuc_filtered"] = mutant_nuc_simple_filtered[idx_sort_MPL][idx]
    df[!, "mutants_AA_fr1"] = mutant_types_set_AA_filtered[1][idx_sort_MPL][idx]
    df[!, "mutants_AA_fr2"] = mutant_types_set_AA_filtered[2][idx_sort_MPL][idx]
    df[!, "mutants_AA_fr3"] = mutant_types_set_AA_filtered[3][idx_sort_MPL][idx]
    df[!, "LoopD"] = vec_LoopD[idx_sort_MPL][idx]
    df[!, "CD4BS"] = vec_CD4contact_binding[idx_sort_MPL][idx]
    df[!, "V1"] = vec_V1[idx_sort_MPL][idx]
    df[!, "V2"] = vec_V2[idx_sort_MPL][idx]
    df[!, "V3"] = vec_V3[idx_sort_MPL][idx]
    df[!, "V4"] = vec_V4[idx_sort_MPL][idx]
    df[!, "V5"] = vec_V5[idx_sort_MPL][idx]    
    df[!, "DH270BS"] = vec_DH270_binding[idx_sort_MPL][idx]
    df[!, "DH272BS"] = vec_DH272_binding[idx_sort_MPL][idx]
    df[!, "DH475BS"] = vec_DH475_binding[idx_sort_MPL][idx]
    df[!, "resist_mut_DH270"] = bool_resist_mut_DH270[idx_sort_MPL][idx]
    df[!, "resist_mut_DH272"] = bool_resist_mut_DH272[idx_sort_MPL][idx]
    df[!, "resist_mut_DH475"] = bool_resist_mut_DH475[idx_sort_MPL][idx]
    df[!, "resist_strain_specific_Abs_CH848"] = bool_resist_strain_specific_Abs_CH848[idx_sort_MPL][idx]
    df[!, "common_mut_SHIV_CH848"] = bool_common_mut_SHIV_CH848[idx_sort_MPL][idx]
    df[!, "reversion"] = reversion_true_false[idx_sort_MPL][idx]
    df[!, "N_linked_glycan_plus_fr1"] = plus_glycan_set[1][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_minus_fr1"] = minus_glycan_set[1][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_shift_fr1"] = shifted_glycan_set[1][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_plus_fr2"] = plus_glycan_set[2][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_minus_fr2"] = minus_glycan_set[2][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_shift_fr2"] = shifted_glycan_set[2][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_plus_fr3"] = plus_glycan_set[3][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_minus_fr3"] = minus_glycan_set[3][idx_sort_MPL][idx]
    df[!, "N_linked_glycan_shift_fr3"] = shifted_glycan_set[3][idx_sort_MPL][idx]
    
    # -- Just copying single site trajectories *-3-poly.csv
    for header_temp in names(csv_selection)
        if(occursin(r"f_at", header_temp) )
            df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
        end
    end

    CSV.write("../out/" *key_RM* "-3-poly_sorted_with_mutant.csv", df);
end

Fr.1: gene: 
Fr.2: gene: vpu, 
Fr.3: gene: env
max: 8372, 8372
min: 6233, 6233
sum(bool_resist_mut_DH270) = 7
sum(bool_resist_mut_DH272) = 2
sum(bool_resist_mut_DH475) = 0
sum(bool_resist_strain_specific_Abs_CH848) = 3
sum(bool_common_mut_SHIV_CH848) = 2
Fr.1: gene: 
Fr.2: gene: vpu, 
Fr.3: gene: env
max: 8375, 8375
min: 6229, 6229
sum(bool_resist_mut_DH270) = 21
sum(bool_resist_mut_DH272) = 4
sum(bool_resist_mut_DH475) = 4
sum(bool_resist_strain_specific_Abs_CH848) = 10
sum(bool_common_mut_SHIV_CH848) = 2
Fr.1: gene: 
Fr.2: gene: vpu, 
Fr.3: gene: env
max: 8366, 8366
min: 6233, 6233
sum(bool_resist_mut_DH270) = 20
sum(bool_resist_mut_DH272) = 5
sum(bool_resist_mut_DH475) = 4
sum(bool_resist_strain_specific_Abs_CH848) = 3
sum(bool_common_mut_SHIV_CH848) = 2
Fr.1: gene: 
Fr.2: gene: vpu, 
Fr.3: gene: env
max: 8375, 8375
min: 6231, 6231
sum(bool_resist_mut_DH270) = 4
sum(bool_resist_mut_DH272) = 4
sum(bool_resist_mut_DH475) = 2
sum(bool_resist_strain_specific_Abs_CH848) = 3
sum(bool_comm

# Processes for Jointed Rhessu

In [113]:
f_dir = "../data/mpl/SHIV.CH848/"
csv_index_and_TF = DataFrame(CSV.File(f_dir * "Marged-RMs-index_with_poly_idx.csv"));
s_MPL_RMs = readdlm(f_dir * "RMs-poly-seq2state-MPL.dat")
s_SL_RMs = readdlm(f_dir * "RMs-poly-seq2state-SL.dat");
dx_RMs = readdlm(f_dir * "numerator-RMs-3-poly-seq2state.dat");
fname_key_RMs_and_Human = ["703010848", "RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720"];
n_max_RMs = length(fname_key_RMs_and_Human);

## Get processed mutations for jointed RMs for CSV file 
(polymorphic_index_global, HXB2_index_global, nucleotide_global, 
    TF_global, TF_global_RMs, nonsynonymous_global, nonsynonymous_global_RMs, 
    edge_gap_global, edge_gap_global_RMs, exposed_global, exposed_global_RMs, 
    flanking_global, flanking_global_RMs, glycan_global, glycan_global_RMs, consensus_global, consensus_global_RMs, 
    dx_marginalized, s_MPL_marginalized, s_SL_marginalized, s_MPL_global, s_SL_global) = get_csv_RMs(
    f_dir, csv_index_and_TF, s_MPL_RMs, s_SL_RMs, dx_RMs, fname_key_RMs_and_Human);

In [114]:
df = DataFrame( 
    polymorphic=polymorphic_index_global, 
    HXB2=HXB2_index_global, 
    nuc=nucleotide_global, 
    TF=TF_global, 
    TF_RMs=TF_global_RMs,
    nonsynonymous=nonsynonymous_global, 
    nonsynonymous_RMs=nonsynonymous_global_RMs, 
    consensus=consensus_global, 
    consensus_RMs=consensus_global_RMs,
    edge_gap=edge_gap_global, 
    edge_gap_RMs=edge_gap_global_RMs, 
    exposed=exposed_global, 
    exposed_RMs=exposed_global_RMs, 
    flanking=flanking_global, 
    flanking_RMs=flanking_global_RMs, 
    glycan=glycan_global, 
    glycan_RMs=glycan_global_RMs,  
)

df[:, "dx_RMs"] = dx_marginalized
df[:, "s_MPL_RMs"] = s_MPL_marginalized
for k in 1:n_max_RMs
    df[:, "s_MPL_"*fname_key_RMs_and_Human[k]] = s_MPL_global[k]
end
df[:, "s_SL_RMs"] = s_SL_marginalized
for k in 1:n_max_RMs
    df[:, "s_SL_"*fname_key_RMs_and_Human[k]] = s_SL_global[k]
end
CSV.write("../out/Marged-RMs-3-poly_SHIVCH848.csv", df);

## Normalizing selection coefficients 

In [115]:
csv_in = DataFrame(CSV.File("../out/Marged-RMs-3-poly_SHIVCH848.csv"));

polymorphic_unique = sort(unique(csv_in.polymorphic))
poly_max = maximum(polymorphic_unique)+1
s_mpl_TF = [0.0 for _ in 1:poly_max]
s_sl_TF = [0.0 for _ in 1:poly_max]
NA_list = [false for _ in 1:poly_max]
for x in polymorphic_unique
    tf_in = custom_mode(csv_in.TF_RMs[csv_in.polymorphic .== x])
    if(tf_in != "NA")
        idx1 = csv_in.polymorphic .== x
        idx2 = csv_in.nuc .== tf_in
        idx = idx1 .* idx2
        s_mpl_TF[x+1] = csv_in.s_MPL_RMs[idx][1]
        s_sl_TF[x+1] = csv_in.s_SL_RMs[idx][1]
        NA_list[x+1] = true
    end
end;

s_MPL_norm = []; s_SL_norm = []
for i in 1:length(csv_in.polymorphic)
    if( NA_list[ csv_in.polymorphic[i] + 1 ] )
        s_mpl_temp = csv_in.s_MPL_RMs[i] - s_mpl_TF[ csv_in.polymorphic[i] + 1 ]
        s_sl_temp = csv_in.s_SL_RMs[i] - s_sl_TF[ csv_in.polymorphic[i] + 1 ]
        push!(s_MPL_norm, s_mpl_temp)
        push!(s_SL_norm, s_sl_temp)
    else
        push!(s_MPL_norm, "NA")
        push!(s_SL_norm, "NA")
    end
end;

In [116]:
idx_non_NA = csv_in.TF_RMs .!= "NA"
df = DataFrame( 
    polymorphic=csv_in.polymorphic[idx_non_NA], 
    HXB2=csv_in.HXB2[idx_non_NA], 
    nuc=csv_in.nuc[idx_non_NA], 
    TF=csv_in.TF[idx_non_NA], 
    TF_RMs=csv_in.TF_RMs[idx_non_NA],
    nonsynonymous=csv_in.nonsynonymous[idx_non_NA], 
    nonsynonymous_RMs=csv_in.nonsynonymous_RMs[idx_non_NA], 
    consensus=csv_in.consensus[idx_non_NA], 
    consensus_RMs=csv_in.consensus_RMs[idx_non_NA],
    edge_gap=csv_in.edge_gap[idx_non_NA], 
    edge_gap_RMs=csv_in.edge_gap_RMs[idx_non_NA], 
    exposed=csv_in.exposed[idx_non_NA], 
    exposed_RMs=csv_in.exposed_RMs[idx_non_NA], 
    flanking=csv_in.flanking[idx_non_NA], 
    flanking_RMs=csv_in.flanking_RMs[idx_non_NA], 
    glycan=csv_in.glycan[idx_non_NA], 
    glycan_RMs=csv_in.glycan_RMs[idx_non_NA],  
)

df[:, "dx_RMs"] = csv_in.dx_RMs[idx_non_NA]
df[:, "s_MPL_RMs"] = s_MPL_norm[idx_non_NA]
for k in 1:n_max_RMs
    df[:, "s_MPL_"*fname_key_RMs_and_Human[k]] = csv_in[idx_non_NA, Symbol("s_MPL_"*fname_key_RMs_and_Human[k])]
end
df[:, "s_SL_RMs"] = s_SL_norm[idx_non_NA]
for k in 1:n_max_RMs
    df[:, "s_SL_"*fname_key_RMs_and_Human[k]] = csv_in[idx_non_NA, Symbol("s_SL_"*fname_key_RMs_and_Human[k])]
end
CSV.write("../out/Marged-RMs-normalized-3-poly_SHIVCH848.csv", df);



# Making CSV file containing mutations for marginalized RMs.  

Note: this process consider only Env gene;

In [117]:
csv_selection = DataFrame(CSV.File("../out/Marged-RMs-normalized-3-poly_SHIVCH848.csv"));
csv_index_and_TF= DataFrame(CSV.File("../data/mpl/SHIV.CH848/Marged-RMs-index_with_poly_idx.csv"));

In [118]:
(vec_gene_idx, vec_nuc_idx, vec_AA_idx, vec_TF, 
    vec_DH270_binding, vec_DH272_binding, vec_DH475_binding, vec_CD4contact_binding, 
    vec_V1, vec_V2, vec_V3, vec_V4, vec_V5, vec_LoopD, 
    mutant_types_set_nuc_simple, mutant_types_set_AA_filtered, 
    Nlinked_plus_set, Nlinked_minus_set, Nlinked_shift_set,
    bool_resist_mut_DH270, bool_resist_mut_DH272, bool_resist_mut_DH475, 
    bool_resist_strain_specific_Abs_CH848, 
    bool_common_mut_SHIV_CH848, reversion_true_false) = get_jointed_RMs_for_CSV_SHIV848(
    csv_selection, csv_index_and_TF, fname_key_RMs_and_Human, fname_key_RMs);
  

gene: env
max: 8375, 8375
min: 6225, 6225
Processing 703010848, done.
Processing RM6163, done.
Processing RM6167, done.
Processing RM6700, done.
Processing RM6713, done.
Processing RM6714, done.
Processing RM6720, done.


In [119]:
# --- Just filling basic information ----#
idx_sort_MPL = sortperm(csv_selection.s_MPL_RMs, rev=true);
idx = mutant_types_set_nuc_simple[idx_sort_MPL] .!= ""
df = DataFrame(
    HXB2=vec_nuc_idx[idx_sort_MPL][idx], 
    AA_idx=replace_nothing_with_missing(vec_AA_idx)[idx_sort_MPL][idx],
    gene=replace_nothing_with_missing(vec_gene_idx)[idx_sort_MPL][idx]
)
df[!, "mutants_nuc"] = mutant_types_set_nuc_simple[idx_sort_MPL][idx]
df[!, "mutants_AA"] = mutant_types_set_AA_filtered[idx_sort_MPL][idx]
# -- Just copying fields already in the *-3-poly.csv
for header_temp in names(csv_selection)
    df[!, header_temp] = csv_selection[:, Symbol(header_temp)][idx_sort_MPL][idx]
end
for k in 1:length(fname_key_RMs_and_Human)
    df[:, "N_plus_"*fname_key_RMs_and_Human[k]] = Nlinked_plus_set[k][idx_sort_MPL][idx]
    df[:, "N_minus_"*fname_key_RMs_and_Human[k]] = Nlinked_minus_set[k][idx_sort_MPL][idx]
    df[:, "N_shift_"*fname_key_RMs_and_Human[k]] = Nlinked_shift_set[k][idx_sort_MPL][idx]
end

# --- Writing computed variables including types of mutaions ----#
df[!, "LoopD"] = vec_LoopD[idx_sort_MPL][idx]
df[!, "CD4BS"] = vec_CD4contact_binding[idx_sort_MPL][idx]
df[!, "V1"] = vec_V1[idx_sort_MPL][idx]
df[!, "V2"] = vec_V2[idx_sort_MPL][idx]
df[!, "V3"] = vec_V3[idx_sort_MPL][idx]
df[!, "V4"] = vec_V4[idx_sort_MPL][idx]
df[!, "V5"] = vec_V5[idx_sort_MPL][idx]
df[!, "DH270BS"] = vec_DH270_binding[idx_sort_MPL][idx]
df[!, "DH272BS"] = vec_DH272_binding[idx_sort_MPL][idx]
df[!, "DH475BS"] = vec_DH475_binding[idx_sort_MPL][idx]
df[!, "resist_mut_DH270"] = bool_resist_mut_DH270[idx_sort_MPL][idx]
df[!, "resist_mut_DH272"] = bool_resist_mut_DH272[idx_sort_MPL][idx]
df[!, "resist_mut_DH475"] = bool_resist_mut_DH475[idx_sort_MPL][idx]
df[!, "resist_strain_specific_Abs_CH848"] = bool_resist_strain_specific_Abs_CH848[idx_sort_MPL][idx]
df[!, "common_mut_SHIV_CH848"] = bool_common_mut_SHIV_CH848[idx_sort_MPL][idx]
df[!, "reversion"] = reversion_true_false[idx_sort_MPL][idx];
CSV.write("../out/RMs-merged-3-poly_sorted_with_mutant_CH848.csv", df);

## Compute enrichment ratios and make a CSV file.

In [5]:
include("../src/process_HIV_mutation_for_CSV.jl") ;

In [18]:
#"""
csv_index_and_TF= DataFrame(CSV.File("../data/mpl/703010848-3-index.csv"));
seq_num_raw = readdlm("../data/mpl/703010848-3-poly-seq2state.dat", Int);
csv_raw_CH848 = DataFrame(CSV.File("../out/703010848-3-poly_sorted_with_mutant.csv"));
#csv_raw_C848 = DataFrame(CSV.File("/net/dali/home/barton/kais/HIV_coevolution_analysis/out/redo_on_cluster/CSV/CH848/703010848-3-poly_sorted_with_mutant.csv"));
categories = ["Rev.", "V1", "V2", "V3", "V4", "V5", "LD", "CD4", "PNG-shield", "PNG-holse", "PNG-shift", "PNG"];
#s_threshold_RMs_set = collect(-0.04:0.005:0.04);
#s_threshold_RMs_set = collect(-0.04:0.0001:0.04);
α_threshold_RMs_set = sort([ [0.01 * i for i in 1:9]; [0.1 * i for i in 1:10] ], rev=true)

@time (x_fold_summary, log_P, α_summary, n_fold_summary, types_summary) = get_enrichment_and_pvalues(csv_raw_CH848, csv_index_and_TF, α_threshold_RMs_set);


 15.659280 seconds (34.38 M allocations: 34.604 GiB, 8.56% gc time, 14.06% compilation time: 5% of which was recompilation)


In [19]:
df = DataFrame(
    categories=types_summary,
    fraction=[@sprintf("%.3f",x) for x in α_summary],
    num_cutoff=Int.(n_fold_summary),
    enrichment=x_fold_summary,
    log10_P = log_P
)
CSV.write("../out/enrichment_CH848_multiply_fraction.csv", df);



# Enrichment SHIV.CH848

In [6]:
fname_key_RMs = ["RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720"];

csv_index_and_TF_accum = DataFrame()
csv_raw_RMs_accum = DataFrame()
for key_RM in fname_key_RMs
    @sprintf("%s\n", key_RM)
    csv_index_and_TF= DataFrame(CSV.File( @sprintf("../data/mpl/SHIV.CH848/%s-3-index.csv", key_RM) ));
    csv_index_and_TF_accum = vcat(csv_index_and_TF_accum, copy(csv_index_and_TF))

    csv_raw_RMs = DataFrame(CSV.File(@sprintf("../out/%s-3-poly_sorted_with_mutant.csv", key_RM)));
    idx_to_take = [!contains(s, "f_at") for s in names(csv_raw_RMs)]
    csv_raw_RMs_accum = vcat(csv_raw_RMs_accum, copy(csv_raw_RMs[:, idx_to_take]))
end
idx_sort = sortperm(csv_raw_RMs_accum.s_MPL, rev=true)
csv_raw_RMs_accum_sorted = copy(csv_raw_RMs_accum[idx_sort, :]);
csv_raw_RMs_accum_sorted.HXB2_index = string.(csv_raw_RMs_accum_sorted.HXB2_index); # because a function that extract integer give error without this treatment. 


In [7]:
idx_non_repeat = []
idx_keep_tracking = []
HXB2_with_repeated = string.(csv_index_and_TF_accum.HXB2)
for x in HXB2_with_repeated
    if(x ∉ idx_keep_tracking)
        push!(idx_non_repeat, true)
        push!(idx_keep_tracking, x)
    else
        push!(idx_non_repeat, false)
    end
end;
csv_index_and_TF_accum_filtered = copy(csv_index_and_TF_accum[Bool.(idx_non_repeat), :]);

In [8]:
categories = ["Rev.", "V1", "V2", "V3", "V4", "V5", "LD", "CD4", "PNG-shield", "PNG-holse", "PNG-shift", "PNG"];
α_threshold_RMs_set = sort([ [0.01 * i for i in 1:9]; [0.1 * i for i in 1:10] ], rev=true)

(x_fold_summary, log_P, α_summary, n_fold_summary, types_summary) = get_enrichment_and_pvalues(csv_raw_RMs_accum_sorted, csv_index_and_TF_accum_filtered, α_threshold_RMs_set);


In [9]:
df = DataFrame(
    categories=types_summary,
    fraction=[@sprintf("%.3f",x) for x in α_summary],
    num_cutoff=Int.(n_fold_summary),
    enrichment=x_fold_summary,
    log10_P = log_P
)
CSV.write("../out/enrichment_grouped_SHIV_CH848_multiply_fraction.csv", df);

